In [1]:
import datetime as datetime
import pandas as pd
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import requests

short_start = "2019-6-01"
med_start = "2019-3-01"
long_start = "2019-01-01"
short_end = "2019-7-01"
med_end = "2019-7-01"
long_end = "2019-7-01"

drop_list = ['XRP' , 'XLM' , 'BCH' , 'LTC' , 'ADA' , 'TRX' , 'EOS' , 'BSV' , 'BNB' , 'XMR' , 'XTZ' ,'DASH' , 'IOT','NEO','ETC']
#long_drop = ['currency' , 'platform' , 'application']

tfs = ['short' , 'med' , 'long']

df_all = dict.fromkeys(tfs)
df_benchmarks = dict.fromkeys(tfs)
returns_all = dict.fromkeys(tfs)
returns_benchmarks = dict.fromkeys(tfs)
results = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
net_return = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
ann_factor = dict.fromkeys(tfs)
total_return = dict.fromkeys(tfs)
percent_sd = dict.fromkeys(tfs)
summary = dict.fromkeys(tfs)
corr = dict.fromkeys(tfs)
betas = dict.fromkeys(tfs)
beta_results = dict.fromkeys(tfs)
total_return_excel = dict.fromkeys(tfs)
sortino_df = dict.fromkeys(tfs)
sortino = dict.fromkeys(tfs)
average_return = dict.fromkeys(tfs)

data_bletchley = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\\bletchley_history.xlsx' , index_col= 0)
data_coins = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bletchley.xlsx' , index_col=0)
#data_bitcoin = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bitcoin.xlsx' , index_col=0)
data_bletchley['ERC20'] = data_bletchley['ethereum']
data_bletchley = data_bletchley.drop('ethereum' , axis=1)
data_bletch = data_bletchley

In [2]:
data_finance = data_bletch.fillna(method='ffill')
data_finance = data_finance.dropna()
index_data = data_finance
data_coins = data_coins.fillna(method='ffill')
data_coins = data_coins.dropna()

#bitcoin_data = data_bitcoin.fillna(method='ffill')
#bitcoin_data = bitcoin_data.dropna()
#bitcoin_data.index = (pd.to_datetime(bitcoin_data.index.values,unit='ms'))
#bitcoin_data.index = bitcoin_data.index.round('d')

index_data.index = (pd.to_datetime(index_data.index.values,unit='ms'))
index_data.index = index_data.index.round('d')
df = index_data.join(data_coins, how='inner')
#df = df.join(bitcoin_data,how='inner')
df

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2018-11-08,740.953740,637.900186,110.287135,311.303644,40.569745,27.086623,24.329500,73.888661,6491.56302,216.05101,...,5.61607,0.07786,6449.50503,9.69729,110.47752,1.31667,164.57949,0.50167,16.62824,9.59420
2018-11-09,721.565444,618.797851,106.479184,303.007814,39.834951,25.967035,23.624929,71.826111,6422.93139,210.55113,...,5.49626,0.07601,102.66811,9.53628,107.71204,1.30120,163.54686,0.48149,16.26065,9.40692
2018-11-10,713.346165,609.780474,103.749804,299.376551,39.246942,25.732236,23.221154,70.634206,6403.41936,209.52185,...,5.39397,0.07435,70.33582,9.54875,107.04013,1.28991,159.66145,0.47727,15.94697,9.49264
2018-11-11,716.586940,610.434094,104.335381,300.675133,39.365448,25.905354,23.253192,70.736600,6389.20007,211.93462,...,5.41039,0.07514,88.33960,9.54135,104.85812,1.30044,161.23425,0.48463,15.93261,9.46828
2018-11-12,719.176084,608.269451,102.901564,301.480361,39.486859,26.000422,22.972163,69.884891,6394.72832,210.50524,...,5.44913,0.07631,114.76690,9.43560,106.21978,1.30966,159.39924,0.48219,16.11287,9.20138
2018-11-13,716.724558,611.223194,101.636233,300.550535,39.208126,26.162344,22.709419,68.951585,6340.79359,209.37507,...,5.39862,0.07553,726.18475,9.31937,104.39349,1.25052,161.92354,0.49305,15.72473,9.12623
2018-11-14,708.465071,596.955295,98.718815,296.715644,38.893893,25.491212,22.114844,66.856327,6254.92565,200.97080,...,5.10496,0.06914,189.55126,8.31798,98.16475,1.16562,149.61548,0.48127,14.69142,8.63940
2018-11-15,639.335345,531.505568,86.586428,267.333560,35.057002,22.916454,19.560126,59.560033,5760.21547,180.95211,...,4.69265,0.06541,173.57708,8.29694,90.42615,1.09946,142.47765,0.43299,13.62862,7.96770
2018-11-16,628.338127,520.364846,84.732955,262.641013,34.171254,22.849624,19.211085,58.138167,5604.88865,179.94259,...,4.66766,0.06312,89.67527,8.07748,87.61144,1.10158,136.64579,0.41837,12.93027,7.72018
2018-11-17,612.375185,503.651023,82.562568,255.850709,33.395493,22.059975,18.680848,56.552413,5567.30291,175.19112,...,4.55622,0.06143,113.27444,7.88014,86.90175,1.10799,136.32234,0.41152,12.75134,7.48791


In [3]:
returns = df.pct_change()
returns = returns.dropna()

In [4]:
#filter master dataframes based on date ranges
returns_all['short'] = returns.loc[short_start:short_end]
returns_benchmarks['short'] = returns.loc[short_start:short_end]
df_all['short'] = df.loc[short_start:short_end]
df_benchmarks['short'] = df.loc[short_start:short_end]

returns_all['med'] = returns.loc[med_start:med_end]
returns_benchmarks['med'] = returns.loc[med_start:med_end]
df_all['med'] = df.loc[med_start:med_end]
df_benchmarks['med'] = df.loc[med_start:med_end]

returns_all['long'] = returns.loc[long_start:long_end]
returns_benchmarks['long'] = returns.loc[long_start:long_end]
df_all['long'] = df.loc[long_start:long_end]
df_benchmarks['long'] = df.loc[long_start:long_end]

#drop coins that aren't benchmarks
for c in drop_list:
    for t in tfs:
        returns_benchmarks[t] = returns_benchmarks[t].drop([c] , axis=1)
        df_benchmarks[t] = df_benchmarks[t].drop([c] , axis=1)

In [5]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-06-25,0.011297,0.014610,0.026951,0.011756,0.012390,0.008003,0.026051,0.020075,0.018503,0.045573
2019-06-26,0.049461,0.004132,0.040379,0.046517,0.059121,0.012253,0.019274,0.011922,0.073959,0.031031
2019-06-27,0.075198,-0.022676,-0.040167,0.067515,0.086314,0.000216,-0.024637,-0.033663,0.061110,-0.069223
2019-06-29,-0.052812,-0.016359,-0.032231,-0.050536,-0.044797,-0.074544,0.002511,-0.007894,-0.011016,-0.027241
2019-06-30,-0.016265,0.041639,0.031614,-0.012294,-0.026277,0.026924,0.077986,0.065812,-0.073015,-0.007090


In [6]:
#drop sectors that don't have data back to 1/1/2018
#for i in long_drop:
#    df_benchmarks['long'] = df_benchmarks['long'].drop([i] , axis=1)
#    returns_benchmarks['long'] = returns_benchmarks['long'].drop([i] , axis=1)

In [7]:
#calculate sortino to btc
for t in tfs:
    sortino_df[t] = pd.DataFrame()
    sortino[t] = pd.DataFrame(index = ['sortino'])
    for col in returns_benchmarks[t]:
        sortino_df[t][col] = returns_benchmarks[t][col] - returns_benchmarks[t]['BTC']
        avg_ret = np.mean(sortino_df[t][col])
        a = [r ** 2 for r in sortino_df[t][col] if r < 0]
        a_sum = np.sum(a)
        b = len(sortino_df[t][col])
        down_risk = np.sqrt(a_sum/b)
        if down_risk > 0.000001:
            sortino[t][col] = (avg_ret / down_risk)*math.sqrt(365)
        else:
            sortino[t][col] = 0

In [8]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-06-25,0.011297,0.014610,0.026951,0.011756,0.012390,0.008003,0.026051,0.020075,0.018503,0.045573
2019-06-26,0.049461,0.004132,0.040379,0.046517,0.059121,0.012253,0.019274,0.011922,0.073959,0.031031
2019-06-27,0.075198,-0.022676,-0.040167,0.067515,0.086314,0.000216,-0.024637,-0.033663,0.061110,-0.069223
2019-06-29,-0.052812,-0.016359,-0.032231,-0.050536,-0.044797,-0.074544,0.002511,-0.007894,-0.011016,-0.027241
2019-06-30,-0.016265,0.041639,0.031614,-0.012294,-0.026277,0.026924,0.077986,0.065812,-0.073015,-0.007090


In [9]:
data_coins_week = data_coins.loc[short_start:short_end]
#data_coins_week = data_coins_week[:]
net_week_return = data_coins_week.pct_change(len(data_coins_week)-1)
ann_week_factor = 365.000/(len(data_coins_week))
data_coins_week_returns = pd.DataFrame()
data_coins_week_returns = data_coins_week.pct_change()
data_coins_week_returns = data_coins_week_returns.dropna()
total_week_return = net_week_return.tail(1)*100
data_coins_week = data_coins_week.append(total_week_return)
data_coins_week.index = data_coins_week.index[:-1].append(pd.Index(['return']))

In [10]:
data_coins_week

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-06-01 00:00:00,8539.342580,268.433010,0.440660,0.13387,114.664680,445.296740,0.033070,8.528800,0.089590,196.706190,32.898020,93.305330,1.511970,165.71767,0.509220,13.60028,8.672400
2019-06-02 00:00:00,8653.909900,269.756840,0.437430,0.13557,115.004230,439.547770,0.038010,7.968970,0.092640,192.482420,34.135640,95.200730,1.490480,167.25949,0.489510,14.03043,8.761450
2019-06-03 00:00:00,8481.012570,256.404320,0.431380,0.12962,107.881160,429.551330,0.036070,7.401290,0.091900,187.396820,30.911610,88.728830,1.445960,154.49649,0.464050,13.15296,9.022030
2019-06-04 00:00:00,8059.153440,242.647390,0.412380,0.12423,101.626760,395.124980,0.032740,6.542430,0.085970,220.039890,29.192360,85.061800,1.335420,147.89546,0.428220,11.85793,9.018630
2019-06-05 00:00:00,7788.867340,244.801480,0.407420,0.12372,103.757890,389.223090,0.035490,6.427060,0.084610,235.707280,29.911350,85.162860,1.230780,147.55333,0.431720,11.69650,8.425930
2019-06-06 00:00:00,7862.608180,246.935120,0.405630,0.12240,104.116830,399.355710,0.034210,6.459230,0.083860,219.953760,30.957530,85.389530,1.196810,147.68800,0.432530,11.64931,8.166730
2019-06-07 00:00:00,7849.246880,254.537150,0.429830,0.12804,113.288370,404.730910,0.033530,6.501470,0.085390,204.113370,32.021450,88.772760,1.345130,147.97210,0.431730,11.97471,8.147970
2019-06-08 00:00:00,7938.355890,248.720350,0.420870,0.12680,116.359280,399.049500,0.033380,6.605510,0.085030,202.682940,31.289150,87.269190,1.308230,148.38446,0.441280,12.13026,8.179180
2019-06-09 00:00:00,7742.689520,238.810630,0.403660,0.12054,114.894570,388.229830,0.031130,6.222610,0.082110,191.047340,30.787940,84.506420,1.246800,140.57765,0.418710,11.64220,8.175270
2019-06-10 00:00:00,7906.981090,240.268000,0.401050,0.12459,119.584990,392.566920,0.031420,6.402060,0.083090,192.790390,31.752990,87.828780,1.258870,149.27090,0.425370,12.07323,8.518470


In [11]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-06-01,0.041348,0.049274,0.048296,0.042092,0.036205,0.070308,0.036536,0.040365,0.006409,0.022189
2019-06-02,-0.006650,-0.012914,-0.006960,-0.007081,-0.003880,-0.027326,0.030779,0.029664,0.013416,0.004932
2019-06-03,0.023113,0.023432,0.036078,0.023397,0.022683,0.026216,0.027430,0.018924,-0.019979,-0.049498
2019-06-04,-0.064283,-0.073058,-0.088001,-0.065366,-0.061202,-0.080995,-0.087987,-0.085338,-0.049742,-0.053653
2019-06-05,-0.053992,-0.068177,-0.051999,-0.054914,-0.057367,-0.056050,-0.039954,-0.034972,-0.033538,0.008877
2019-06-06,0.015249,0.032071,0.011633,0.016298,0.014904,0.015340,0.032412,0.036527,0.009467,0.008716
2019-06-07,0.000269,0.010669,0.006954,0.001104,-0.004544,0.015853,-0.003658,0.002689,-0.001699,0.030786
2019-06-08,0.028403,0.033563,0.043694,0.029062,0.033018,0.022003,0.037444,0.036937,0.011353,-0.022852
2019-06-09,-0.013569,-0.010486,0.009351,-0.012896,-0.010274,-0.022106,-0.004127,-0.007080,-0.024648,-0.039843
2019-06-10,-0.036700,-0.031546,-0.048082,-0.036584,-0.031771,-0.046035,-0.049098,-0.043343,0.021219,0.006103


In [12]:
print ('short start: ', returns_benchmarks['short'].index[0])
print ('medium start: ', returns_benchmarks['med'].index[0])
print ('long start: ', returns_benchmarks['long'].index[0])
print ('end: ', returns_benchmarks['short'].index[-2])
print ('end: ', returns_benchmarks['med'].index[-2])
print ('end: ', returns_benchmarks['long'].index[-2])

short start:  2019-06-01 00:00:00
medium start:  2019-03-01 00:00:00
long start:  2019-01-01 00:00:00
end:  2019-06-29 00:00:00
end:  2019-06-29 00:00:00
end:  2019-06-29 00:00:00


In [13]:
returns_benchmarks['short'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-06-01,0.041348,0.049274,0.048296,0.042092,0.036205,0.070308,0.036536,0.040365,0.006409,0.022189
2019-06-02,-0.006650,-0.012914,-0.006960,-0.007081,-0.003880,-0.027326,0.030779,0.029664,0.013416,0.004932
2019-06-03,0.023113,0.023432,0.036078,0.023397,0.022683,0.026216,0.027430,0.018924,-0.019979,-0.049498
2019-06-04,-0.064283,-0.073058,-0.088001,-0.065366,-0.061202,-0.080995,-0.087987,-0.085338,-0.049742,-0.053653
2019-06-05,-0.053992,-0.068177,-0.051999,-0.054914,-0.057367,-0.056050,-0.039954,-0.034972,-0.033538,0.008877


In [14]:
print (data_coins_week.head(n=1))
data_coins_week.tail(n=1)

                            BTC        ETH      XRP      XLM        LTC  \
2019-06-01 00:00:00  8539.34258  268.43301  0.44066  0.13387  114.66468   

                           BCH      TRX     EOS      ADA        BSV       BNB  \
2019-06-01 00:00:00  445.29674  0.03307  8.5288  0.08959  196.70619  32.89802   

                          XMR      XTZ       DASH      IOT       NEO     ETC  
2019-06-01 00:00:00  93.30533  1.51197  165.71767  0.50922  13.60028  8.6724  


,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
return,36.190802,12.625724,-7.647619,-19.13797,10.649653,-5.325235,-0.967644,-30.013366,-4.788481,2.686413,0.686272,0.806363,-34.796987,-4.13551,-21.957111,27.22466,-10.129607


In [15]:
annualize = math.sqrt(365)
for t in tfs:
    results[t] = returns_benchmarks[t].describe()
    vol[t] = pd.DataFrame()
    vol[t] = results[t].loc['std']*annualize*100
    vol[t].name = 'annualized volatility'
    results[t] = results[t].append(df_benchmarks[t].iloc[0])
    results[t].index = results[t].index[:-1].append(pd.Index(['open price']))
    results[t] = results[t].append(df_benchmarks[t][-1:])
    results[t].index = results[t].index[:-1].append(pd.Index(['last price']))
    results[t] = results[t].append(vol[t])

In [16]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-06-25,0.011297,0.014610,0.026951,0.011756,0.012390,0.008003,0.026051,0.020075,0.018503,0.045573
2019-06-26,0.049461,0.004132,0.040379,0.046517,0.059121,0.012253,0.019274,0.011922,0.073959,0.031031
2019-06-27,0.075198,-0.022676,-0.040167,0.067515,0.086314,0.000216,-0.024637,-0.033663,0.061110,-0.069223
2019-06-29,-0.052812,-0.016359,-0.032231,-0.050536,-0.044797,-0.074544,0.002511,-0.007894,-0.011016,-0.027241
2019-06-30,-0.016265,0.041639,0.031614,-0.012294,-0.026277,0.026924,0.077986,0.065812,-0.073015,-0.007090


In [17]:
for t in tfs:
    average_return[t] = pd.DataFrame()
    average_return[t] = returns_benchmarks[t]
    average_return[t].loc['mean'] = returns_benchmarks[t].mean(axis=0)
    returns_benchmarks[t] = returns_benchmarks[t][:-1]

In [18]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-06-01 00:00:00,0.041348,0.049274,0.048296,0.042092,0.036205,0.070308,0.036536,0.040365,0.006409,0.022189
2019-06-02 00:00:00,-0.006650,-0.012914,-0.006960,-0.007081,-0.003880,-0.027326,0.030779,0.029664,0.013416,0.004932
2019-06-03 00:00:00,0.023113,0.023432,0.036078,0.023397,0.022683,0.026216,0.027430,0.018924,-0.019979,-0.049498
2019-06-04 00:00:00,-0.064283,-0.073058,-0.088001,-0.065366,-0.061202,-0.080995,-0.087987,-0.085338,-0.049742,-0.053653
2019-06-05 00:00:00,-0.053992,-0.068177,-0.051999,-0.054914,-0.057367,-0.056050,-0.039954,-0.034972,-0.033538,0.008877
2019-06-06 00:00:00,0.015249,0.032071,0.011633,0.016298,0.014904,0.015340,0.032412,0.036527,0.009467,0.008716
2019-06-07 00:00:00,0.000269,0.010669,0.006954,0.001104,-0.004544,0.015853,-0.003658,0.002689,-0.001699,0.030786
2019-06-08 00:00:00,0.028403,0.033563,0.043694,0.029062,0.033018,0.022003,0.037444,0.036937,0.011353,-0.022852
2019-06-09 00:00:00,-0.013569,-0.010486,0.009351,-0.012896,-0.010274,-0.022106,-0.004127,-0.007080,-0.024648,-0.039843
2019-06-10 00:00:00,-0.036700,-0.031546,-0.048082,-0.036584,-0.031771,-0.046035,-0.049098,-0.043343,0.021219,0.006103


In [19]:
for t in tfs:
    net_return[t] = df_benchmarks[t].pct_change(len(df_benchmarks[t])-1)
    ann_factor[t] = 365.000/(len(df_benchmarks[t]))
    total_return[t] = net_return[t].tail(1)*100
    total_return_excel[t] = net_return[t].tail(1)
    percent_sd[t] = results[t].iloc[2,:]*100
    results[t] = results[t].append(percent_sd[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['daily volatility']))
    #print total_return_excel[t]
    results[t] = results[t].append(total_return_excel[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['return']))
    #print results[t]
    #print average_return[t]/(percent_sd[t]*0.01).transpose()
    Sharpe_values = (average_return[t].loc['mean']/(percent_sd[t]*0.01))*annualize
    Sharpe_values = Sharpe_values.rename('sharpe_values')
    results[t] = results[t].append(Sharpe_values)
    results[t].index = results[t].index[:-1].append(pd.Index(['Sharpe']))
    results[t] = results[t].append(sortino[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['Sortino']))
#print (results[t])

In [20]:
returns_benchmarks['med'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-03-01 00:00:00,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.001505,0.001286
2019-03-02 00:00:00,0.001945,0.033469,0.021862,0.004711,0.003317,0.005152,0.032925,0.036816,-0.006676,-0.000766
2019-03-03 00:00:00,-0.002430,-0.000707,-0.000303,-0.002244,0.001227,-0.009133,0.001460,0.000615,0.000186,-0.024998
2019-03-04 00:00:00,-0.011021,-0.021609,-0.008489,-0.011707,-0.009177,-0.013162,-0.019840,-0.021733,-0.012855,-0.024823
2019-03-05 00:00:00,-0.024378,-0.029798,-0.037056,-0.025141,-0.021542,-0.042697,-0.015249,-0.012223,0.038668,-0.000925


In [21]:
for t in tfs:
    summary[t] = results[t].iloc[8:]
    corr[t] = returns_benchmarks[t].corr(method='spearman')
    betas[t] = returns_benchmarks[t].cov()
    beta_results[t] = pd.DataFrame()
    beta_results[t] = betas[t].iloc[0,:] / betas[t].iloc[3,0]
    corr[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_corr'+t+'.csv')
    summary[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_results'+t+'.csv')
beta_all = pd.DataFrame()
beta_all['MTD'] = beta_results['short']
beta_all['QTD'] = beta_results['med']
beta_all['YTD'] = beta_results['long']

In [22]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
2019-06-25 00:00:00,0.011297,0.014610,0.026951,0.011756,0.012390,0.008003,0.026051,0.020075,0.018503,0.045573
2019-06-26 00:00:00,0.049461,0.004132,0.040379,0.046517,0.059121,0.012253,0.019274,0.011922,0.073959,0.031031
2019-06-27 00:00:00,0.075198,-0.022676,-0.040167,0.067515,0.086314,0.000216,-0.024637,-0.033663,0.061110,-0.069223
2019-06-29 00:00:00,-0.052812,-0.016359,-0.032231,-0.050536,-0.044797,-0.074544,0.002511,-0.007894,-0.011016,-0.027241
2019-06-30 00:00:00,-0.016265,0.041639,0.031614,-0.012294,-0.026277,0.026924,0.077986,0.065812,-0.073015,-0.007090


In [23]:
summary['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,904.975523,898.926786,96.815645,380.925777,50.857877,29.575051,33.068381,101.570238,8539.342580,268.433010
last price,1160.152554,981.829040,95.868577,481.167532,68.348928,29.404806,40.001116,118.018603,11629.799130,302.324620
annualized volatility,66.458295,104.265549,129.309336,65.470264,67.572784,88.485304,111.180987,104.548994,62.085080,62.240537
daily volatility,3.478586,5.457508,6.768360,3.426870,3.536921,4.631532,5.819479,5.472344,3.249682,3.257819
return,0.281971,0.092224,-0.009782,0.263153,0.343920,-0.005756,0.209648,0.161941,0.361908,0.126257
Sharpe,5.819762,2.137672,0.976127,5.625460,6.528094,1.310929,3.091187,2.778004,6.725808,3.157079
Sortino,-0.866465,-2.610803,-3.100212,-1.362192,0.770207,-4.502960,-0.944066,-1.658548,0.000000,-3.752777


In [24]:
pd.set_option('display.max_columns', 500)
summary['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,424.914158,381.398019,57.301770,178.648490,22.516762,15.764553,17.515368,50.875261,3807.077940,135.471300
last price,1160.152554,981.829040,95.868577,481.167532,68.348928,29.404806,40.001116,118.018603,11629.799130,302.324620
annualized volatility,69.449496,83.448653,93.325290,68.286956,70.031148,81.594835,81.540861,80.839871,63.011008,59.996447
daily volatility,3.635153,4.367902,4.884869,3.574303,3.665598,4.270869,4.268044,4.231352,3.298147,3.140358
return,1.730322,1.574290,0.673047,1.693376,2.035469,0.865248,1.283773,1.319764,2.054784,1.231651
Sharpe,4.863161,3.956004,2.189656,4.873892,5.297100,2.784743,3.621922,3.669939,5.853842,4.487310
Sortino,-0.840174,-0.689564,-2.396584,-0.959041,0.062379,-2.509095,-1.235096,-1.224919,0.000000,-1.955344


In [25]:
summary['long']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
open price,420.511599,337.862280,54.644247,175.290083,22.237102,15.347474,13.168780,39.924454,3771.967010,134.012200
last price,1160.152554,981.829040,95.868577,481.167532,68.348928,29.404806,40.001116,118.018603,11629.799130,302.324620
annualized volatility,68.760438,80.299211,85.749344,67.970061,67.412968,82.318020,78.444364,78.741881,56.885881,66.189174
daily volatility,3.599086,4.203053,4.488326,3.557716,3.528556,4.308722,4.105966,4.121538,2.977543,3.464500
return,1.758907,1.906004,0.754413,1.744979,2.073644,0.915938,2.037572,1.956048,2.083219,1.255948
Sharpe,3.276602,3.035659,1.651717,3.290579,3.674662,1.903153,3.238522,3.136857,4.379497,2.799024
Sortino,-0.623837,-0.101422,-1.757053,-0.659962,-0.040838,-1.678962,0.090205,-0.039181,0.000000,-1.317571


In [26]:
corr['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.501970,0.527586,0.985714,0.994089,0.697044,0.395567,0.426601,0.654680,0.298522
twenty,0.501970,1.000000,0.872906,0.571429,0.468473,0.901478,0.933990,0.956650,0.057635,0.233005
forty,0.527586,0.872906,1.000000,0.588177,0.506404,0.845813,0.834483,0.809852,0.068966,0.228079
total,0.985714,0.571429,0.588177,1.000000,0.974384,0.749754,0.470936,0.503448,0.632512,0.283744
currency,0.994089,0.468473,0.506404,0.974384,1.000000,0.661576,0.372414,0.401478,0.672414,0.282759
platform,0.697044,0.901478,0.845813,0.749754,0.661576,1.000000,0.783251,0.812808,0.206404,0.248768
application,0.395567,0.933990,0.834483,0.470936,0.372414,0.783251,1.000000,0.973399,0.006404,0.100493
ERC20,0.426601,0.956650,0.809852,0.503448,0.401478,0.812808,0.973399,1.000000,0.045320,0.186207
BTC,0.654680,0.057635,0.068966,0.632512,0.672414,0.206404,0.006404,0.045320,1.000000,0.347291
ETH,0.298522,0.233005,0.228079,0.283744,0.282759,0.248768,0.100493,0.186207,0.347291,1.000000


In [27]:
corr['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.660957,0.654940,0.987869,0.984055,0.843636,0.623102,0.594674,0.523678,0.448201
twenty,0.660957,1.000000,0.833581,0.737558,0.639872,0.840122,0.918858,0.923242,0.204514,0.390917
forty,0.654940,0.833581,1.000000,0.713947,0.631562,0.821779,0.819082,0.788368,0.202228,0.328321
total,0.987869,0.737558,0.713947,1.000000,0.974179,0.880974,0.690090,0.666682,0.493961,0.437980
currency,0.984055,0.639872,0.631562,0.974179,1.000000,0.779893,0.583165,0.559240,0.559150,0.386863
platform,0.843636,0.840122,0.821779,0.880974,0.779893,1.000000,0.786427,0.756508,0.265244,0.476225
application,0.623102,0.918858,0.819082,0.690090,0.583165,0.786427,1.000000,0.984310,0.215956,0.394896
ERC20,0.594674,0.923242,0.788368,0.666682,0.559240,0.756508,0.984310,1.000000,0.223598,0.418191
BTC,0.523678,0.204514,0.202228,0.493961,0.559150,0.265244,0.215956,0.223598,1.000000,0.355048
ETH,0.448201,0.390917,0.328321,0.437980,0.386863,0.476225,0.394896,0.418191,0.355048,1.000000


In [28]:
corr['long']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
ten,1.000000,0.737745,0.736545,0.991764,0.987053,0.888325,0.682651,0.659337,0.452096,0.553117
twenty,0.737745,1.000000,0.835996,0.795437,0.719131,0.871001,0.924026,0.926393,0.231388,0.491650
forty,0.736545,0.835996,1.000000,0.778717,0.717259,0.852833,0.816838,0.788383,0.245851,0.456778
total,0.991764,0.795437,0.778717,1.000000,0.979552,0.915580,0.736012,0.715911,0.433271,0.547040
currency,0.987053,0.719131,0.717259,0.979552,1.000000,0.835802,0.653426,0.633549,0.489680,0.500638
platform,0.888325,0.871001,0.852833,0.915580,0.835802,1.000000,0.802770,0.779575,0.266283,0.581433
application,0.682651,0.924026,0.816838,0.736012,0.653426,0.802770,1.000000,0.985170,0.250213,0.477150
ERC20,0.659337,0.926393,0.788383,0.715911,0.633549,0.779575,0.985170,1.000000,0.251608,0.487788
BTC,0.452096,0.231388,0.245851,0.433271,0.489680,0.266283,0.250213,0.251608,1.000000,0.364335
ETH,0.553117,0.491650,0.456778,0.547040,0.500638,0.581433,0.477150,0.487788,0.364335,1.000000


In [29]:
beta_all

,MTD,QTD,YTD
ten,1.025163,1.021025,1.014557
twenty,0.657258,0.785101,0.858324
forty,0.727783,0.836726,0.866429
total,1.000000,1.000000,1.000000
currency,1.033216,1.018475,0.984284
platform,0.961080,1.006917,1.075700
application,0.562572,0.691666,0.752902
ERC20,0.511561,0.663854,0.738070
BTC,0.675214,0.634884,0.522799
ETH,0.230046,0.347849,0.548963


In [30]:
coins_high_low = data_coins_week_returns.describe()
coins_high_low

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,0.011530,0.004652,-0.002111,-0.006496,0.005233,-0.001305,0.002173,-0.010921,-0.000994,0.003264,0.003108,0.002518,-0.013484,0.000256,-0.007399,0.011050,-0.002630
std,0.041142,0.033455,0.035635,0.039790,0.058508,0.034586,0.070100,0.050245,0.037691,0.069253,0.076090,0.065376,0.048620,0.057551,0.046980,0.074588,0.044735
min,-0.095989,-0.069223,-0.117900,-0.147072,-0.190987,-0.080145,-0.231061,-0.180447,-0.103875,-0.209388,-0.212341,-0.245258,-0.138222,-0.211748,-0.157625,-0.205996,-0.181940
25%,-0.005933,-0.007090,-0.014576,-0.020716,-0.014931,-0.019067,-0.030828,-0.026379,-0.020636,-0.026421,-0.016688,-0.021297,-0.032837,-0.031291,-0.038706,-0.025893,-0.014929
50%,0.018503,0.008877,-0.001322,-0.004790,0.010909,0.001442,0.011838,0.005005,0.001179,0.003002,0.008043,0.010995,-0.015065,0.002787,-0.000329,0.012599,0.004242
75%,0.029917,0.023766,0.014786,0.018944,0.038796,0.012416,0.031867,0.022084,0.023509,0.024323,0.037620,0.039621,0.011816,0.029418,0.017980,0.032459,0.025105
max,0.093995,0.072190,0.059785,0.046078,0.088089,0.073143,0.149380,0.047135,0.068112,0.174192,0.206948,0.136769,0.123929,0.120170,0.058506,0.202462,0.041980


In [31]:
returns_benchmarks['short'].describe()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,0.010596,0.006106,0.003458,0.010090,0.012086,0.003178,0.009416,0.007957,0.011440,0.005384
std,0.034786,0.054575,0.067684,0.034269,0.035369,0.046315,0.058195,0.054723,0.032497,0.032578
min,-0.064283,-0.129733,-0.187915,-0.065366,-0.061202,-0.082141,-0.133114,-0.118127,-0.073015,-0.069223
25%,-0.006650,-0.017878,-0.032231,-0.011257,-0.004544,-0.022106,-0.020096,-0.011934,-0.005933,-0.007090
50%,0.015410,0.009649,0.006954,0.015103,0.014904,0.015210,0.014012,0.011700,0.013416,0.008877
75%,0.031554,0.032071,0.033772,0.037917,0.033018,0.024755,0.032412,0.032781,0.022003,0.023766
max,0.075198,0.201843,0.244920,0.067515,0.086314,0.122272,0.212115,0.203677,0.073959,0.072190


In [32]:
data_coins_week.tail(n=1)

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
return,36.190802,12.625724,-7.647619,-19.13797,10.649653,-5.325235,-0.967644,-30.013366,-4.788481,2.686413,0.686272,0.806363,-34.796987,-4.13551,-21.957111,27.22466,-10.129607
